In [2]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import folium
from folium import plugins
import geopandas as gpd
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LogColorMapper, ColorBar
from bokeh.palettes import YlOrRd
import json

In [3]:
shapefile = 'data/geo_data/ne_110m_admin_0_countries.shp'
#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
#Rename columns.
gdf.columns = ['country', 'country_code', 'geometry']
gdf = gdf.drop(gdf.index[159]) #remove antartica
gdf.head()

DriverError: data/geo_data/ne_110m_admin_0_countries.shp: No such file or directory

In [ ]:
datafile = 'time_series_covid19_deaths_global_iso3_regions.csv'
#Read csv file using pandas
df = pd.read_csv(datafile)
df.head()

In [ ]:
 df.groupby(['ISO 3166-1 Alpha 3-Codes']).sum().reset_index().head()

In [ ]:
 df.groupby(['ISO 3166-1 Alpha 3-Codes']).sum().reset_index()
country_total = df.groupby(['ISO 3166-1 Alpha 3-Codes']).sum().iloc[:,-1]
country_total = pd.DataFrame(country_total).reset_index()

In [ ]:
country_total.head()

In [ ]:
now = '4/23/20'
df_newest = pd.DataFrame(country_total[['ISO 3166-1 Alpha 3-Codes', now]])
df_newest.columns = ['country_code', 'deaths']
df_newest = df_newest.drop(0)

In [ ]:
df_newest.head()

In [ ]:
merged = gdf.merge(df_newest, on = 'country_code')
merged.head()

In [ ]:
#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

In [ ]:
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = YlOrRd[9]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LogColorMapper(palette = palette, low = 0, high = 6e4)

#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

#Create color bar. 
#color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
#border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

color_bar = ColorBar(color_mapper=color_mapper, location=(0,0), orientation='horizontal', padding=0)


#Create figure object.
p = figure(title = 'Newest Death count', plot_height = 600 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'deaths', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')

#Display figure inline in Jupyter Notebook.
output_notebook()

#Display figure.
show(p)

In [ ]:

country_total = df.groupby(['ISO 3166-1 Alpha 3-Codes']).sum().reset_index()


In [ ]:
from bokeh.io import curdoc, output_notebook
from bokeh.models import DateSlider, HoverTool, CustomJS, LogTicker, BasicTickFormatter
from bokeh.layouts import widgetbox, row, column
from datetime import date
import colorcet as cc
#Define function that returns json_data for year selected by user.


def json_data():
    merged = gdf.merge(country_total, left_on = 'country_code', right_on ='ISO 3166-1 Alpha 3-Codes', how = 'left')
    merged.fillna('No data', inplace = True)
    merged['selected'] = merged['4/23/20']
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data())
#Define a sequential multi-hue color palette.
palette = cc.fire[:-25]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LogColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
#The reason for not setting low = 0 is that we are using log scale and log(0) is undefined and this causes problems in
#the colorbar
color_mapper = LogColorMapper(palette = palette, low = 1, high = 5e6, low_color = '#fff0ba', nan_color = '#d9d9d9')
#Define custom tick labels for color bar.
tick_labels = {'1': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#Add hover tool
hover = HoverTool(tooltips = [ ('Country','@country'),('% Death', '@selected')])
#Create color bar. 
#color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(), label_standoff=8,width = 600, height = 20,
#                     border_line_color=None,location = (0,0), orientation = 'horizontal')

color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0), formatter = BasicTickFormatter(power_limit_high = 10))

#Create figure object.
p = figure(title = 'Death count', plot_height = 400 , plot_width = 600, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.visible = False
p.add_layout(color_bar, 'right')
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'selected', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify layout
p.add_layout(color_bar, 'below')

def getDate(number):
    print('test')
    if (number <= 31): #january
        return '1/'+str(number)+'/20'
    number = number - 31
    if (number <= 29): #february
        return '2/' + str(number) + '/20'
    number = number - 29
    if (number <= 31):
        return '3/' + str(number) + '/20'
    number = number - 31
    return '4/' + str(number) + '/20'


callback = CustomJS(args=dict(source=geosource), code="""
    console.log(cb_obj.value)
    var d = new Date(cb_obj.value);
    console.log(d)
    var day = d.getDate()
    var month = d.getMonth()+1
    var date = month + '/' + day + '/20'
    console.log(date)
    
    d = source.attributes.data
    d['selected'] = d[date]
    source.attributes.data = d
    
    source.change.emit();
""")
                    
                    
# Make a slider object: slider 
slider = DateSlider(title = 'Date',start=date(2020, 1, 22), end=date(2020, 4, 23),
                    value=date(2020, 4, 23), step = 1, width=p.plot_width-100)
slider.callback_policy = "mouseup"
slider.js_on_change('value', callback)


# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)
#Display plot inline in Jupyter notebook
#output_notebook()
output_file('map.html')
#Display plot
show(layout)